In [1]:
import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


### Exercice 1 – Alignement de textes
On vous demande dans cet exercice de programmer une version récursive en mode préfixe, puis la version Bottom-up de l’alignement de texte par programmation dynamique, puis de faire en sorte de comparer le résultat d’une approche greedy et de votre algorithme sur l’exemple donné en cours

In [25]:
def Badness(words, text_width):
    line_length = sum(len(word) for word in words) + len(words) - 1

    if line_length > text_width:
        bad = float('inf')
    else:
        bad = (text_width - line_length)**3

    return bad

###### Recursive - Suffix ######################################################
def JustifDP_rec_Suffix(words, i, text_width, seb):
    if i == 69: # final case
        return 0
    if i in seb: # if the destination has already been calculated
        return seb[i] # returns from memory
    
    mini=float('inf') # inicialize inf

    for j in range(i+1, len(words)+1): # Test all possible divisions starting from the first word
        val=Badness(words[i:j],text_width)+JustifDP_rec_Suffix(words, j, text_width, seb)
        if val <= mini: # Updates the lowest value
            mini = val
            seb[i] = mini

    return mini


###### Recursive - Preffix #####################################################
def JustifDP_rec_Preffix(words, i, text_width, seb):
    if i == 0: # final case (first word)
        return 0
    if i in seb:
        return seb[i]
    
    mini=float('inf')

    for j in range(i-1, -1, -1): # Test all possible divisions starting from the last word
        val=Badness(words[j:i],text_width)+JustifDP_rec_Preffix(words, j, text_width, seb)
        if val <= mini:
            mini = val
            seb[i] = mini

    return mini


###### Bottom Up #####################################################
def JustifDP_BU(words, text_width):
    
    seb = []

    for w in range(len(words)+1):
        seb.append(float('inf'))
    seb[0] = 0

    for i in range(1, len(words)+1):
        for j in range(i-1, -1, -1):
            val=Badness(words[j:i],text_width)+seb[j]
            if val <= seb[i]:
                seb[i] = val

    return seb[len(words)]



def JustifGreedy(words, text_width):
    total_cost = 0
    i = 0

    # Processa o texto linha por linha
    while i < len(words):
        j = i + 1
        while j <= len(words) and Badness(words[i:j], text_width) != float('inf'):
            j += 1
        # Adiciona o custo da linha atual ao total
        total_cost += Badness(words[i:j-1], text_width)
        # Move para a próxima linha
        i = j - 1

    return total_cost
    


In [26]:
text_width=30
text2split="Mignonne allons voir si la rose Qui ce matin avoit desclose Sa \
robe de pourpre au Soleil a point perdu ceste vesprée Les plis de sa robe \
pourprée Et son teint au vostre pareil Las voyez comme en peu d'espace \
Mignonne elle a dessus la place Las las ses beautez laissé cheoir O vrayment \
marastre Nature Puis qu'une telle fleur ne dure Que du matin jusques au soir !"
words=text2split.split()

res = {}
resDP_Suffix = JustifDP_rec_Suffix(words, 0, text_width, res)
print(resDP_Suffix)

res = {}
resDP_Preffix = JustifDP_rec_Preffix(words, len(words), text_width, res)
print(resDP_Preffix)

resDP_BU = JustifDP_BU(words, text_width)
print(resDP_BU)

resGreedy = JustifGreedy(words, text_width)
print(resGreedy)

897
897
897
1125


In [30]:
def reconstruct_lines(words, text_width, justify_func, start_idx, seb):
    """
    Reconstrói as linhas após calcular o custo da justificação.
    """
    lines = []
    while start_idx < len(words):
        best_end = None
        mini = float('inf')
        for j in range(start_idx + 1, len(words) + 1):
            cost = Badness(words[start_idx:j], text_width) + (
                justify_func(words, j, text_width, seb) if justify_func != JustifDP_rec_Preffix else seb[j]
            )
            if cost < mini:
                mini = cost
                best_end = j
        lines.append(words[start_idx:best_end])
        start_idx = best_end
    return lines


def reconstruct_lines_BU(words, text_width, justify_func):
    """
    Reconstrói as linhas usando a abordagem Bottom-Up.
    """
    seb = [float('inf')] * (len(words) + 1)
    seb[0] = 0
    splits = [-1] * (len(words) + 1)

    for i in range(1, len(words) + 1):
        for j in range(i - 1, -1, -1):
            cost = Badness(words[j:i], text_width) + seb[j]
            if cost < seb[i]:
                seb[i] = cost
                splits[i] = j

    lines = []
    idx = len(words)
    while idx > 0:
        prev = splits[idx]
        lines.append(words[prev:idx])
        idx = prev

    return lines[::-1]

def reconstruct_lines_Greedy(words, text_width):
    """
    Reconstrói as linhas após calcular o custo da justificação.
    """
    lines = []
    while start_idx < len(words):
        best_end = None
        mini = float('inf')
        for j in range(start_idx + 1, len(words) + 1):
            cost = Badness(words[start_idx:j], text_width) + (
                justify_func(words, j, text_width, seb) if justify_func != JustifDP_rec_Preffix else seb[j]
            )
            if cost < mini:
                mini = cost
                best_end = j
        lines.append(words[start_idx:best_end])
        start_idx = best_end
    return lines


def JustifyText(words, text_width, justify_func):
    """
    Aplica a função de justificação escolhida e retorna o texto em linhas.

    Args:
        words (list): Lista de palavras a serem justificadas.
        text_width (int): Largura máxima permitida para uma linha.
        justify_func (function): Função que realiza a justificação.

    Returns:
        list: Texto separado em linhas, cada linha sendo uma lista de palavras.
    """
    seb = {}
    if justify_func == JustifDP_rec_Suffix:
        return reconstruct_lines(words, text_width, justify_func, 0, seb)
    elif justify_func == JustifDP_rec_Preffix:
        return reconstruct_lines(words, text_width, justify_func, len(words), seb)
    elif justify_func == JustifDP_BU:
        return reconstruct_lines_BU(words, text_width, justify_func)
    else:
        raise ValueError("Função de justificação inválida!")


In [8]:
# Escolha a função de justificação
justified_lines = JustifyText(words, text_width, JustifDP_BU)

# Exibe as linhas justificadas
for line in justified_lines:
    print(" ".join(line))



Mignonne allons voir si la
rose Qui ce matin avoit
desclose Sa robe de pourpre
au Soleil a point perdu ceste
vesprée Les plis de sa robe
pourprée Et son teint au
vostre pareil Las voyez comme
en peu d'espace Mignonne elle
a dessus la place Las las
ses beautez laissé cheoir O
vrayment marastre Nature Puis
qu'une telle fleur ne dure Que
du matin jusques au soir !


In [33]:
# Loop para justificar texto usando diferentes métodos
methods = [
    ("Recursivo de Sufixo", JustifDP_rec_Suffix),
    ("Recursivo de Prefixo", JustifDP_rec_Preffix),
    ("Bottom-Up", JustifDP_BU),
    ("Greedy", JustifGreedy)
]

for method_name, method_func in methods:
    print(f"\n--- {method_name} ---")
    
    if method_func in [JustifDP_rec_Suffix, JustifDP_rec_Preffix]:
        seb = {}
        justified_lines = JustifyText(words, text_width, method_func)
        cost = method_func(words, 0 if method_func == JustifDP_rec_Suffix else len(words), text_width, seb)
    elif method_func == JustifDP_BU:
        seb = {}
        justified_lines = JustifyText(words, text_width, method_func)
        cost = method_func(words, text_width)
    else:  # Greedy
        justified_lines = reconstruct_lines_BU(words, text_width, method_func)
        cost = method_func(words, text_width)

    # Exibe o texto dividido em linhas
    for line in justified_lines:
        print(" ".join(line))
    
    # Exibe o custo total
    print(f"Custo total: {cost}")



--- Recursivo de Sufixo ---
Mignonne allons voir si
la rose Qui ce matin avoit
desclose Sa robe de pourpre
au Soleil a point perdu ceste
vesprée Les plis de sa robe
pourprée Et son teint au
vostre pareil Las voyez comme
en peu d'espace Mignonne elle
a dessus la place Las las
ses beautez laissé cheoir O
vrayment marastre Nature Puis
qu'une telle fleur ne dure
Que du matin jusques au soir !
Custo total: 897

--- Recursivo de Prefixo ---
Custo total: 897

--- Bottom-Up ---
Mignonne allons voir si la
rose Qui ce matin avoit
desclose Sa robe de pourpre
au Soleil a point perdu ceste
vesprée Les plis de sa robe
pourprée Et son teint au
vostre pareil Las voyez comme
en peu d'espace Mignonne elle
a dessus la place Las las
ses beautez laissé cheoir O
vrayment marastre Nature Puis
qu'une telle fleur ne dure Que
du matin jusques au soir !
Custo total: 897

--- Greedy ---
Mignonne allons voir si la
rose Qui ce matin avoit
desclose Sa robe de pourpre
au Soleil a point perdu ceste
vesprée Les plis d

### Exercice 2 – Parenthétisation
On vous demande dans cet exercice de programmer une version récursive du problème de placement de parenthèses à partir de la récurrence définie dans le cours, puis de tester votre algorithme sur l’exemple donné en cours.

In [19]:
def Parent_Dp(i, j, parent, memo):
    # Se o resultado já foi calculado, retorna diretamente
    if (i, j) in memo:
        return memo[(i, j)]
    
    mini = float('inf')
    ind = -1

    # Divide o problema em subproblemas menores
    for k in range(i, j):
        new = Parent_Dp(i, k, parent, memo) + Parent_Dp(k + 1, j, parent, memo) + d[i] * d[k + 1] * d[j + 1]
        if new < mini:
            mini = new
            ind = k

    # Salva os resultados no memo e parent
    memo[(i, j)] = mini
    parent[(i, j)] = ind
    return mini

In [22]:
# Dimensões das matrizes
d = [13, 5, 89, 3, 34]

# Inicialização de estruturas auxiliares
parent1 = {}
memo = {}

# Caso base: custo zero para uma única matriz
for i in range(len(d) - 1):
    memo[(i, i)] = 0
    
result = Parent_Dp(0, len(d) - 2, parent1, memo)

print("Custo mínimo de parentetização:", result)

Custo mínimo de parentetização: 2856


In [23]:
def Parent_BU(d):
    n = len(d) - 1  # Número de matrizes
    memo = {}  # Tabela de custos mínimos
    parent = {}  # Tabela de parentesco (melhor divisão)

    # Inicializa os custos para subproblemas triviais (uma única matriz)
    for i in range(n):
        memo[(i, i)] = 0

    # Preenche a tabela de forma crescente no tamanho da cadeia
    for l in range(2, n + 1):  # l é o tamanho da subcadeia
        for i in range(n - l + 1):
            j = i + l - 1
            memo[(i, j)] = float('inf')  # Inicializa com infinito
            for k in range(i, j):  # Testa todas as divisões possíveis
                cost = memo[(i, k)] + memo[(k + 1, j)] + d[i] * d[k + 1] * d[j + 1]
                if cost < memo[(i, j)]:
                    memo[(i, j)] = cost
                    parent[(i, j)] = k

    # Retorna o custo mínimo e as tabelas de memorização e parentesco
    return memo[(0, n - 1)]

In [24]:
# Executa o algoritmo Bottom-Up
min_cost = Parent_BU(d)

# Resultados
print("Custo mínimo de parentetização:", min_cost)

Custo mínimo de parentetização: 2856
Tabela de memorização (memo): {(0, 0): 0, (1, 1): 0, (2, 2): 0, (3, 3): 0, (0, 1): 5785, (1, 2): 1335, (2, 3): 9078, (0, 2): 1530, (1, 3): 1845, (0, 3): 2856}
Tabela de parentesco (parent): {(0, 1): 0, (1, 2): 1, (2, 3): 2, (0, 2): 0, (1, 3): 2, (0, 3): 2}
